In [ ]:
from numpy import array
from keras.callbacks import CSVLogger
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import GRU
from keras.layers import Embedding
import io

In [ ]:
# load into memory
def load_file(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text
in_filename = r'Trigram_data_3_1.txt'
dataset = load_file(in_filename)
Data_lines = dataset.split('\n')

In [5]:
Data_lines

NameError: name 'Data_lines' is not defined

In [ ]:
# integer encode sequences of words
token = Tokenizer()
token.fit_on_texts(Data_lines) 
seq=token.texts_to_sequences(Data_lines)
word_size=len(token.word_index)+1
word_size

In [4]:
seq

NameError: name 'seq' is not defined

In [ ]:
length = 4

In [3]:
seq = array(seq, dtype=object)
seq

NameError: name 'array' is not defined

In [1]:
a=1
h=0
while(a!=0):
    limit=seq.size
    d =[]
    a=0
    for i in range(0 , limit):
        l=len(seq[i])
        if(l!=length):
            print(seq[i])
            seq = np.delete(seq, i, 0)
            
            a=1
            print("error in ", i)
            print("Deleting") 
            h = h+1
            break        
print("Total : ", h)     

NameError: name 'seq' is not defined

In [2]:
seq=np.array(seq.tolist())
seq

NameError: name 'np' is not defined

In [55]:
seq.shape

(58585, 4)

In [ ]:
# separate into input and output
seq = array(seq)
X, y = seq[:,:-1], seq[:,-1]
y = to_categorical(y, num_classes=word_size)
seq_len = X.shape[1]

In [ ]:
# Define data generator
def data_generator(X_data, y_data, batch_size):
    num_batches=len(X_data) // batch_size
    while True:
        for i in range(num_batches):
            start_idx=i*batch_size
            end_idx=(i+1)*batch_size
            yield X_data[start_idx:end_idx], y_data[start_idx:end_idx]

# Define model
model = Sequential()
model.add(Embedding(word_size, 100, input_length=seq_len))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(GRU(256, return_sequences=True))
model.add(Dropout(0.1))
model.add(GRU(256))
model.add(Dense(256, activation='relu'))
model.add(Dense(word_size, activation='softmax')) 
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']
csv_logger=CSVLogger('data_3_1.log')
batch_size=128
epochs=150
train_generator=data_generator(X,y,batch_size)
history=model.fit(train_generator, steps_per_epoch=len(X) // batch_size, epochs=epochs, callbacks=[csv_logger])

In [ ]:
# save the model to file
model.save('model_3_1.h5')
dump(token, open(r'token_3_1.pkl', 'wb'))

In [ ]:
# draw graph
from matplotlib import pyplot
pyplot.plot(history.history['loss'])
pyplot.plot(history.history['accuracy'])
pyplot.title('model loss and accuracy ')
pyplot.xlabel('epoch')
pyplot.legend(['loss', 'accuracy'], loc='upper right')
pyplot.show()